In [0]:
# Imports
import numpy as np
from PIL import Image
import requests
from io import BytesIO
from tqdm import tqdm
import csv
from datetime import datetime

from keras import backend
from keras.models import Model
from keras.applications.vgg16 import VGG16

from scipy.optimize import fmin_l_bfgs_b

import tensorflow as tf

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Hyperparams
ITERATIONS = 20
CHANNELS = 3
IMAGE_SIZE = 500
IMAGE_WIDTH = IMAGE_SIZE
IMAGE_HEIGHT = IMAGE_SIZE
IMAGENET_MEAN_RGB_VALUES = [123.68, 116.779, 103.939]
CONTENT_WEIGHT = 0.02 #default = 0.02
STYLE_WEIGHT = 0 #default = 4.5
TOTAL_VARIATION_WEIGHT = 0.995 #default = 0.995 
TOTAL_VARIATION_LOSS_FACTOR = 1.25 #default = 1.25

In [0]:
# Paths
input_image_path = "drive/My Drive/AIschool Team4/장지선/input.png"
style_image_path = "drive/My Drive/AIschool Team4/장지선/style.png"
output_image_path = "drive/My Drive/AIschool Team4/장지선/output.png"
# combined_image_path = "drive/My Drive/AIschool Team4/장지선/combined.png"

# San Francisco
san_francisco_image_path = "https://www.economist.com/sites/default/files/images/print-edition/20180602_USP001_0.jpg"

# Warsaw by Tytus Brzozowski, http://t-b.pl
tytus_image_path = "http://meetingbenches.com/wp-content/flagallery/tytus-brzozowski-polish-architect-and-watercolorist-a-fairy-tale-in-warsaw/tytus_brzozowski_13.jpg"


In [0]:
#Input visualization 
input_image = Image.open(BytesIO(requests.get(san_francisco_image_path).content))
input_image = input_image.resize((IMAGE_WIDTH, IMAGE_HEIGHT))
input_image.save(input_image_path)
input_image

In [0]:
# Style visualization 
style_image = Image.open(BytesIO(requests.get(tytus_image_path).content))
style_image = style_image.resize((IMAGE_WIDTH, IMAGE_HEIGHT))
style_image.save(style_image_path)
style_image

In [0]:
# Data normalization and reshaping from RGB to BGR
input_image_array = np.asarray(input_image, dtype="float32")
input_image_array = np.expand_dims(input_image_array, axis=0)
input_image_array[:, :, :, 0] -= IMAGENET_MEAN_RGB_VALUES[2]
input_image_array[:, :, :, 1] -= IMAGENET_MEAN_RGB_VALUES[1]
input_image_array[:, :, :, 2] -= IMAGENET_MEAN_RGB_VALUES[0]
input_image_array = input_image_array[:, :, :, ::-1]

style_image_array = np.asarray(style_image, dtype="float32")
style_image_array = np.expand_dims(style_image_array, axis=0)
style_image_array[:, :, :, 0] -= IMAGENET_MEAN_RGB_VALUES[2]
style_image_array[:, :, :, 1] -= IMAGENET_MEAN_RGB_VALUES[1]
style_image_array[:, :, :, 2] -= IMAGENET_MEAN_RGB_VALUES[0]
style_image_array = style_image_array[:, :, :, ::-1]

In [0]:
# Model
input_image = backend.variable(input_image_array)
style_image = backend.variable(style_image_array)
combination_image = backend.placeholder((1, IMAGE_HEIGHT, IMAGE_SIZE, 3))

input_tensor = backend.concatenate([input_image,style_image,combination_image], axis=0)
model = VGG16(input_tensor=input_tensor, include_top=False)
model.summary()

In [0]:
def content_loss(content, combination):
    return backend.sum(backend.square(combination - content))
  
def gram_matrix(x):
    features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))
    gram = backend.dot(features, backend.transpose(features))
    return gram

def compute_style_loss(style, combination):
    style = gram_matrix(style)
    combination = gram_matrix(combination)
    size = IMAGE_HEIGHT * IMAGE_WIDTH
    return backend.sum(backend.square(style - combination)) / (4. * (CHANNELS ** 2) * (size ** 2))
  
def total_variation_loss(x):
    a = backend.square(x[:, :IMAGE_HEIGHT-1, :IMAGE_WIDTH-1, :] - x[:, 1:, :IMAGE_WIDTH-1, :])
    b = backend.square(x[:, :IMAGE_HEIGHT-1, :IMAGE_WIDTH-1, :] - x[:, :IMAGE_HEIGHT-1, 1:, :])
    return backend.sum(backend.pow(a + b, TOTAL_VARIATION_LOSS_FACTOR))

def evaluate_loss_and_gradients(x):
    x = x.reshape((1, IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
    outs = backend.function([combination_image], outputs)([x])
    loss = outs[0]
    gradients = outs[1].flatten().astype("float64")
    return loss, gradients
  
def visualx(x):
    x = x.copy()
    x = x.reshape((IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
    x = x[:, :, ::-1]
    x[:, :, 0] += IMAGENET_MEAN_RGB_VALUES[2]
    x[:, :, 1] += IMAGENET_MEAN_RGB_VALUES[1]
    x[:, :, 2] += IMAGENET_MEAN_RGB_VALUES[0]
    x = np.clip(x,0,255).astype('uint8')
    output_image = Image.fromarray(x)
    return output_image
    
    
def toCSV(csv_col, dict_):
    file = open('drive/My Drive/AIschool Team4/장지선/loss.csv', 'w', newline = '')
    csvfile = csv.DictWriter(file, fieldnames=csv_col)
    for row in dict_:
        csvfile.writerow(row)
    file.close()

class Evaluator:

    def loss(self, x):
        loss, gradients = evaluate_loss_and_gradients(x)
        self._gradients = gradients
        return loss

    def gradients(self, x):
        return self._gradients

In [0]:
conv_layer = []
for layer in model.layers:
    conv_layer.append(layer.name)
    print(layer.name)
del conv_layer[0]
conv_layer

In [0]:
!ls drive/My\ Drive/AIschool\ Team4/장지선/output/

In [0]:
layers = dict([(layer.name, layer.output) for layer in model.layers])

conv_layer = ['block2_conv2','block3_conv1']

num =0
csv_col = ['block', 'iteration', 'loss']
dict_data = []

for i in tqdm(conv_layer):
#     merged_summary = tf.summary.merge_all()
    logdir = 'drive/My Drive/AIschool Team4/장지선/logs'
    file_writer = tf.contrib.summary.create_file_writer(logdir, name=datetime.now().strftime('%Y%m%d-%H%M%S'))
#     sess.run(tf.global_variables_initializer())

    content_layer = str(i)
    num += 1
    print("trial",num,content_layer)
    layer_features = layers[content_layer]
    content_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]

    loss = backend.variable(0.)
    loss = loss + CONTENT_WEIGHT * content_loss(content_image_features,
                                          combination_features)

    style_layers = ["block1_conv2", "block2_conv2", "block3_conv3", "block4_conv3", "block5_conv3"]

    for layer_name in style_layers:
        layer_features = layers[layer_name]
        style_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        style_loss = compute_style_loss(style_features, combination_features)
        loss = loss + (STYLE_WEIGHT / len(style_layers)) * style_loss


    loss = loss + TOTAL_VARIATION_WEIGHT * total_variation_loss(combination_image)

    outputs = [loss]
    outputs += backend.gradients(loss, combination_image)

    evaluator = Evaluator()

    x = np.random.uniform(0, 255, (1, IMAGE_HEIGHT, IMAGE_WIDTH, 3)) - 128.

    for i in tqdm(range(ITERATIONS)):
        x, loss, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.gradients, maxfun=20)
        canvas = visualx(x)
        path = 'drive/My Drive/AIschool Team4/장지선/ParameterChange/outputs'+content_layer+'_total0.998_iteration_'+str(i)+'.png'
        canvas.save(str(path))
        dict_data.append({csv_col[0]:content_layer ,csv_col[1]:i ,csv_col[2]:loss })
        xx = x.copy()
        xx = xx.reshape(IMAGE_HEIGHT, IMAGE_WIDTH, 3)
        xx = tf.expand_dims(xx, axis=0)
        xx = tf.transpose(xx, perm=[3,1,2,0])
        tf.summary.image('input',xx, 10)

        print("Iteration %d completed with loss %d" % (i, loss))
            
toCSV(csv_col, dict_data) 


In [0]:
# # Visualizing combined results
# combined = Image.new("RGB", (IMAGE_WIDTH*3, IMAGE_HEIGHT))
# x_offset = 0
# for image in map(Image.open, [input_image_path, style_image_path, output_image_path]):
#     combined.paste(image, (x_offset, 0))
#     x_offset += IMAGE_WIDTH
# combined.save(combined_image_path)
# combined

In [0]:
crop = Image.new("RGB",(IMAGE_WIDTH, IMAGE_HEIGHT))
area = (50,50,200,200)
for i in conv_layer:
    imgs_path = 'drive/My Drive/AIschool Team4/장지선/output/'+i+'/content_iteration_19.png'
    crop = Image.open(imgs_path)
    crop = crop.crop(area)
    crop.save('drive/My Drive/AIschool Team4/장지선/output/'+i+'/crop_withtotal_19.png')
